<a href="https://colab.research.google.com/github/JuanJoseMV/neuraltextgen/blob/main/Example_finetuning_dante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!git clone --recursive https://github.com/JuanJoseMV/neuraltextgen.git
!pip install -r /content/neuraltextgen/texygen/requirements.txt
!pip install simpletransformers

In [2]:
!git clone https://github.com/dlang/druntime.git


Cloning into 'druntime'...
remote: Enumerating objects: 49306, done.
remote: Counting objects: 100% (1342/1342), done.
remote: Compressing objects: 100% (534/534), done.
remote: Total 49306 (delta 870), reused 1134 (delta 784), pack-reused 47964
Receiving objects: 100% (49306/49306), 19.90 MiB | 15.73 MiB/s, done.
Resolving deltas: 100% (32730/32730), done.


Define the model (bert-base-italian-uncased)

In [3]:
import torch
import os
import numpy as np
import re
import pandas as pd
from transformers import BertTokenizer


os.chdir("/content/neuraltextgen/")
from NeuralTextGenerator import BertTextGenerator, FormatTokenizer


it_bert_model = BertTextGenerator("dbmdz/bert-base-italian-uncased")
tokenizer = it_bert_model.tokenizer
model = it_bert_model.model
device = it_bert_model.device

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Preproces manually the text replacing '\n' with unused0 and masking 3 tokens per sentence

In [4]:
import re

tercet_pattern = re.compile('(\n[\w,.-;’\'\"àèéìòù\d«» ]+){3}\n')

lines = []
labels = []
labels_idx = []

with open('/content/druntime/benchmark/extra-files/dante.txt', 'r') as f:
    for match in tercet_pattern.finditer(''.join(f.readlines())):
        line = match[0]

        line = line.replace('\n', ' unused0 ')
      
        tokens = tokenizer.encode(line,add_special_tokens=False, return_tensors ='np')[0]
        token_to_mask_idx = np.random.randint(1, len(tokens)-1, 3)
        
        labels.append(tokens[token_to_mask_idx])
        tokens[token_to_mask_idx] = tokenizer.mask_token_id
        
        labels_idx.append(token_to_mask_idx)
        
        lines.append(tokenizer.decode(tokens))

labels = torch.tensor(labels)
labels_idx = torch.tensor(labels_idx)

In [5]:
print('Masked line')
print(lines[0])
print(f'   - masked token id: {labels[0]}')
print(f'   - masked token: {tokenizer.convert_ids_to_tokens(labels[0])}')
print(f'   - masked index: {labels_idx[0]}')

Masked line
unused0 nel mezzo del cammin di nostra vita unused0 [MASK] ritrovai [MASK] una selva oscura, unused0 che [MASK] diritta via era smarrita. unused0
   - masked token id: tensor([151, 143, 246])
   - masked token: ['per', 'la', 'mi']
   - masked index: tensor([12, 19,  9])


In [6]:
labels_idx

tensor([[12, 19,  9],
        [13, 24,  6],
        [24, 24, 36],
        ...,
        [30, 28, 30],
        [32, 11,  7],
        [12, 12, 27]])

In [7]:
it_bert_model.finetune(lines, labels, labels_idx, epochs=6, lr=5e-5)

unused0 nel mezzo del cammin di nostra vita unused0 [MASK] ritrovai [MASK] una selva oscura, unused0 che [MASK] diritta via era smarrita. unused0
unused0 tant ’ e ama [MASK] che poco e piu morte ; [MASK] ma per trattar del ben ch ’ i ’ [MASK] trovai, unused0 diro de l ’ altre cose ch ’ i ’ v ’ ho scorte. unused0
unused0 io non so ben ridir com ’ i ’ v ’ intrai, unused0 tant ’ era pien di [MASK] a quel punto unused0 che la verace via abbandonai [MASK] unused0
unused0 ma poi ch ’ i ’ fui al pie d ’ un [MASK] giunto, unused0 la dove terminava quella [MASK] unused0 [MASK] m ’ avea di paura il cor compunto, unused0
unused0 guardai in alto e vidi le sue spalle [MASK] vestite [MASK] de [MASK] raggi del pianeta unused0 che mena dritto altrui per ogne calle. unused0
unused0 allor fu la [MASK] un poco queta, unused0 che [MASK] lago del cor m ’ era durata unused0 la notte ch ’ i ’ passai con tanta pieta [MASK] unused0
unused0 e come quei che con lena affannata, unused0 uscito fuor del pela [MASK]

In [9]:
%%time

parameters = {'n_sentences': 1,  # 1000
              'batch_size': 1,  # 50
              'avg_len':100,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
  for k,v in it_bert_model.formatter.dict_token_replace.items():
    s = s.replace(v, k)

  print(s)


 la vendetta, ch ’ elli fa certo, ch ’ ella dice 
 disse ; « quivi dieda, ma di quaa 
 tal men come uccelletto ; per ch ’ il veder ti vuol bene, 
 perche come maestro avvienta e di quella, si comincio ’ maestro, « or aspetta si sen lavi si che le due lingue, 
 ma ’ le genti care cosi come faran li scalghi non tiranni, 

CPU times: user 6.11 s, sys: 14.2 ms, total: 6.13 s
Wall time: 6.17 s


In [10]:
%%time

parameters = {'n_sentences': 3,  # 1000
              'batch_size': 3,  # 50
              'avg_len':20,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'parallel',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
  for k,v in it_bert_model.formatter.dict_token_replace.items():
    s = s.replace(v, k)

  print(s)


 muover le sue gambe 
 parlar divino

 verra la ove la dove ’ avra peccato 


 guarda e vedi 
 come l ’ occhio vede, 

CPU times: user 5.68 s, sys: 19.1 ms, total: 5.7 s
Wall time: 5.71 s


In [12]:
%%time

parameters = {'n_sentences': 3,  # 1000
              'batch_size': 3,  # 50
              'avg_len':40,
              'max_len':1000,
              'std_len' : 3,
              # 'top_k': 100,
              'generation_method':'attention',
              'temperature': 1,
              'sample': True,
              # 'burnin': 450,
              'max_iter': 500,
              'seed_text': "",
              }


sents = it_bert_model.generate(save_to_path='/content/prova.txt', **parameters)

for s in sents:
  for k,v in it_bert_model.formatter.dict_token_replace.items():
    s = s.replace(v, k)

  print(s)


 « ivi mi benga ancor 
 molto temendo te prima, quando si che trasga », pero elli laocoron tal 
 veder bisanzio mi fu sposa ; 


 « virtute mio valor si fa », 
 ch ’ i ’ udiva a l ’ una un ’ altra, 
 di mala d ’ ogne tanti ammassi tuoi » 


 ma non volsi prima a te 
 fare tutto questo, che solo a veder si di veder bene, 
 fu di fess modo tutto per lui ancor non nascondei. 

CPU times: user 9.27 s, sys: 13.9 ms, total: 9.29 s
Wall time: 9.26 s


Note how in this manner the model learns automatically where to place '\n' (the structure of the tercets)